# Analysis of AGL J1736-3250 Flares

Copyright (c) 2025, INAF - Istituto Nazionale di Astrofisica

Authors: Andrea Bulgarelli, Gabriele Panebianco

Licensed under the BSD 3-Clause License.
See LICENSE file for details.

Steps:
- Read the Data and Set the Configuration
- Set the Model Hypothesis
- Generate the AGILE Maps
- Perform the MLE to get the results of Table 2 of the paper.
- Estimate the Light Curve with 6h, 4h, 2h bins to get the results of Table 3 of the paper.

In [ ]:
from agilepy.api.AGAnalysis import AGAnalysis
from agilepy.utils.AstroUtils import AstroUtils

import os
import yaml

In [ ]:
# Initialize the configuration file and Set the Time Intervals

# Set only one of the following time intervals.
# MJD_start, MJD_stop = 54343, 54344  # Flare E01
MJD_start, MJD_stop = 54732, 54733  # Flare E02
MJD_start, MJD_stop = 54746, 54747  # Flare E03
MJD_start, MJD_stop = 54915, 54916  # Flare E04
MJD_start, MJD_stop = 54935, 54936  # Flare E05
MJD_start, MJD_stop = 55100, 55101  # Flare E06
MJD_start, MJD_stop = 55109, 55110  # Flare E07
MJD_start, MJD_stop = 55112, 55113  # Flare E08
MJD_start, MJD_stop = 55599, 55600  # Flare E09
MJD_start, MJD_stop = 56182, 56183  # Flare E10
MJD_start, MJD_stop = 56199, 56200  # Flare E11
MJD_start, MJD_stop = 56351, 56352  # Flare E12
MJD_start, MJD_stop = 57786, 57787  # Flare E13
MJD_start, MJD_stop = 57823, 57824  # Flare E14
MJD_start, MJD_stop = 58199, 58200  # Flare E15
MJD_start, MJD_stop = 58368, 58369  # Flare E16
MJD_start, MJD_stop = 58744, 58745  # Flare E17
MJD_start, MJD_stop = 59656, 59657  # Flare E18
MJD_start, MJD_stop = 59806, 59807  # Flare E19

baseconfFilePath = f"/home/flareadvocate/workspace/IGRJ17354-3255/AGILE_detection_AGLJ1736-3250/configuration/config_analysis.yaml"
confFilePath = f"/home/flareadvocate/workspace/IGRJ17354-3255/AGILE_detection_AGLJ1736-3250/configuration/config_analysis_MJD_{MJD_start}_{MJD_stop}.yaml"

In [ ]:
# Modify configuration file with new time intervals and output directory

# Load YAML
with open(baseconfFilePath, "r") as f:
    data = yaml.safe_load(f)

# Modify fields
data["output"]["outdir"]  = f"/home/flareadvocate/workspace/IGRJ17354-3255/AGILE_detection_AGLJ1736-3250/results/MJD_{MJD_start}_{MJD_stop}/"
data["selection"]["tmin"] = MJD_start
data["selection"]["tmax"] = MJD_stop

# Save modified YAML
with open(confFilePath, "w") as f:
    yaml.dump(data, f, sort_keys=False)


In [ ]:
# Define the AGAnalysis object and set the correct Options
ag = AGAnalysis(confFilePath)
ag.setOptionTimeMJD(tmin=MJD_start, tmax=MJD_stop)
ag.setOptionEnergybin(0) # 100 MeV - 10 GeV
ag.setOptions(evtfile=f"/home/flareadvocate/workspace/IGRJ17354-3255/AGILE_detection_AGLJ1736-3250/data/MJD_{MJD_start}_{MJD_stop}/EVT.index",
              logfile=f"/home/flareadvocate/workspace/IGRJ17354-3255/AGILE_detection_AGLJ1736-3250/data/MJD_{MJD_start}_{MJD_stop}/LOG.index"
              )

In [ ]:
# Show the current configuration
ag.printOptions()

### Set Model Hypotheses

In [ ]:
# 2AGL catalog sources within 15 degrees
sources = ag.loadSourcesFromCatalog("2AGL", rangeDist = (0, 15), show=False)

# Add new source AGLJ1736-3250
newSourceDict = {
    "glon" : 355.447,
    "glat": -0.2689,
    "spectrumType" : "PowerLaw",
    "flux": 2.0e-07,
    "index": 2.1
}
newSource = ag.addSource("AGLJ1736-3250", newSourceDict)

# Free flux and position of AGLJ1736-3250
affectedSources = ag.freeSources('name == "AGLJ1736-3250"',"flux", True, show=False)
affectedSources = ag.freeSources('name == "AGLJ1736-3250"', "pos", True, show=False)

# Print Model
sources = ag.selectSources("flux > 0", show = True)

### Run Analysis

In [ ]:
# Maps
maplistfile = ag.generateMaps()

# Print
print(f"Map list file: {maplistfile}")

In [ ]:
# Maximum Likelihood Estimation on the full period
mlemaplist = ag.mle()

# Print results
_source = ag.selectSources('name == "AGLJ1736-3250"', show=True)

# Write source regions on file
regFile = ag.writeSourcesOnFile("source_regions", "reg")
print(f"Write: {regFile}")

In [ ]:
# Extract results for AGLJ1736-3250
multiSqrtTS   = _source[0].get("multiSqrtTS")['value']
multiFlux     = _source[0].get("multiFlux")['value']
multiFluxErr  = _source[0].get("multiFluxErr")['value']
multiExp      = _source[0].get("multiExp")['value']
multiGalCoeff = _source[0].get("multiGalCoeff")['value']
multiIsoCoeff = _source[0].get("multiIsoCoeff")['value']
multiCounts   = _source[0].get("multiCounts")['value']
multiCountsErr= _source[0].get("multiCountsErr")['value']

print(f"Analysis for MJD {MJD_start} - {MJD_stop} completed.")
print(f" Exposure = {multiExp:.3e} cm2 s.")
print(f" GAL coeff = {multiGalCoeff}, ISO coeff = {multiIsoCoeff}")
print(f" Sqrt(TS) = {multiSqrtTS:.3g}")
print(f" Flux = ({multiFlux:.3g} +/- {multiFluxErr:.3g}) ph/s/cm2")
print(f" Counts = {multiCounts:.1f} +/- {multiCountsErr:.1f}")

In [ ]:
# Show Counts Map
ag.displayCtsSkyMaps(maplistFile=maplistfile, smooth=3,
                     regFiles=[regFile], regFileColors=["green"],
                     # catalogRegions="2AGL", catalogRegionsColor="blue",
                     saveImage=True
                    )

In [ ]:
# Show Exposure Map
ag.displayExpSkyMaps(maplistFile=maplistfile, smooth=3,
                     regFiles=[regFile], regFileColors=["green"],
                     # catalogRegions="2AGL", catalogRegionsColor="blue",
                     saveImage=True
                    )

In [ ]:
# Show Gas Map
ag.displayGasSkyMaps(maplistFile=maplistfile, smooth=3,
                     regFiles=[regFile], regFileColors=["green"],
                     # catalogRegions="2AGL", catalogRegionsColor="blue",
                     saveImage=True
                    )

In [ ]:
# Show Intensity Map
ag.displayIntSkyMaps(maplistFile=maplistfile, smooth=3,
                     regFiles=[regFile], regFileColors=["green"],
                     # catalogRegions="2AGL", catalogRegionsColor="blue",
                     saveImage=True
                    )

## Light Curve Estimation: Intra-day Flares

In [ ]:
# Background estimation for intra-day LightCurve
gal, iso, lcmaplist = ag.calcBkg("AGLJ1736-3250", pastTimeWindow = 0)
print(f"isocoeff: {ag.getOption('isocoeff')} galcoeff: {ag.getOption('galcoeff')}")
# Set the coefficients for  the Light Curve Analysis
ag.setOptions(galcoeff=gal, isocoeff=iso, galmode2=0, isomode2=0)

Light Curve: 6h

In [ ]:
# Light Curve: 6h
lightCurveData6h = ag.lightCurveMLE("AGLJ1736-3250",
                                    tmin=MJD_start, tmax=MJD_stop, timetype="MJD",
                                    binsize=3600*6)

In [ ]:
print(f"Light Curve File 6h: {lightCurveData6h}")

# Plot
ag.displayLightCurve("mle", saveImage=False)

# Save and rename plot
lc_plot = ag.displayLightCurve("mle", saveImage=True)
new_path = os.path.join(os.path.dirname(lc_plot), 'LightCurve_6h.png')
os.rename(lc_plot, new_path)

Light Curve: 4h

In [ ]:
# Light Curve: 4h
lightCurveData4h = ag.lightCurveMLE("AGLJ1736-3250",
                                    tmin=MJD_start, tmax=MJD_stop, timetype="MJD",
                                    binsize=3600*4)

In [ ]:
print(f"Light Curve File 4h: {lightCurveData4h}")

# Plot
ag.displayLightCurve("mle", saveImage=False)

# Save and rename plot
lc_plot = ag.displayLightCurve("mle", saveImage=True)
new_path = os.path.join(os.path.dirname(lc_plot), 'LightCurve_4h.png')
os.rename(lc_plot, new_path)

Light Curve: 2h

In [ ]:
# Light Curve: 2h
lightCurveData2h = ag.lightCurveMLE("AGLJ1736-3250",
                                    tmin=MJD_start, tmax=MJD_stop, timetype="MJD",
                                    binsize=3600*2)

In [ ]:
print(f"Light Curve File 2h: {lightCurveData2h}")

# Plot
ag.displayLightCurve("mle", saveImage=False)

# Save and rename plot
lc_plot = ag.displayLightCurve("mle", saveImage=True)
new_path = os.path.join(os.path.dirname(lc_plot), 'LightCurve_2h.png')
os.rename(lc_plot, new_path)

In [ ]:
print(f"Analysis MJD {MJD_start} - {MJD_stop} completed!")